In [1]:
# Circa 13 minuti per eseguire sta cella
!pip install --pre -U unified-planning

!rm -rf up-pyperplan
!git clone https://github.com/aiplan4eu/up-pyperplan
!pip install up-pyperplan/

!rm -rf up-fast-downward
!git clone https://github.com/aiplan4eu/up-fast-downward
!pip install up-fast-downward/

!apt-get install openjdk-17-jdk
!rm -rf up-enhsp
!git clone https://github.com/aiplan4eu/up-enhsp.git
!pip install up-enhsp/

!rm -rf up-tamer
!git clone https://github.com/aiplan4eu/up-tamer
!pip install up-tamer/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.6/641.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.4 MB/s eta 0:00:00
Cloning into 'up-pyperplan'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 548 (delta 131), reused 168 (delta 102), pack-reused 245
Receiving objects: 100% (548/548), 100.04 KiB | 3.85 MiB/s, done.
Resolving deltas: 100% (294/294), done.
Processing ./up-pyperplan
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 1.6 MB/s eta 0:00:00
  Created wheel for up-pyperplan: filename=up_pyperplan-1.0.0.1.dev1-py3-none-any.whl size=12510 sha256=583526008f6230c7d1ee103293093f08075ece64336b2f26ad8e7eb0a8c9bc89
  Stored in directory: /root/.cache/pip/wheels/5d/d3/4e/8f12b3847d96de7b7a4afeed40e814f5eafc552d4bde579c67
Successfully built up-pyperplan
Cloning into 'up-fast-downward'...
rem

In [79]:
from unified_planning.shortcuts import *
# from unified_planning.model.multi_agent import *
from unified_planning.engines import PlanGenerationResultStatus
import numpy as np
import random
import copy
import timeit

In [136]:
class Slide_tile():
    def __init__(self, n_col, n_row):
        self.n_col = n_col
        self.n_row = n_row
        self.tiles, self.b_x, self.b_y = self.__generate_tiles()
        self.shuffle()

    def __generate_tiles(self):
        tiles = []
        for i in range(self.n_row):
            row = []
            for j in range(self.n_col):
                if i == self.n_row-1 and j == self.n_col-1:
                    row.append("-")
                else:
                    row.append(str(i*self.n_row + j))
            tiles.append(row)
        return tiles, i, j

    def shuffle(self):
        for i in range(1000):
            r = random.randint(0,3)
            if r == 0:
                self.move_up()
            if r == 1:
                self.move_down()
            if r == 2:
                self.move_right()
            if r == 3:
                self.move_left()

    def move_down(self):
        if self.b_x != 0:
            tmp = self.tiles[self.b_x-1][self.b_y]
            self.tiles[self.b_x-1][self.b_y] = "-"
            self.tiles[self.b_x][self.b_y] = tmp
            self.b_x -= 1

    def move_up(self):
        if self.b_x != self.n_row-1:
            tmp = self.tiles[self.b_x+1][self.b_y]
            self.tiles[self.b_x+1][self.b_y] = "-"
            self.tiles[self.b_x][self.b_y] = tmp
            self.b_x += 1

    def move_right(self):
        if self.b_y != 0:
            tmp = self.tiles[self.b_x][self.b_y-1]
            self.tiles[self.b_x][self.b_y-1] = "-"
            self.tiles[self.b_x][self.b_y] = tmp
            self.b_y -= 1

    def move_left(self):
        if self.b_y != self.n_col-1:
            tmp = self.tiles[self.b_x][self.b_y+1]
            self.tiles[self.b_x][self.b_y+1] = "-"
            self.tiles[self.b_x][self.b_y] = tmp
            self.b_y += 1

    def __str__(self):
        s = ""
        for i in range(self.n_row):
            for j in range(self.n_col):
                s += self.tiles[i][j] + " "
            s += "\n"
        s = s[:-1]
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
TIMEOUT = 5
class Slide_tile_PDDL():
    def __init__(self, difficult="easy", optimal_plan = False, verbose = True):
        self.verbose = verbose
        self.optimal_plan = optimal_plan
        # Slide tile init
        if difficult.lower() == "easy":
            self.slide_tile = Slide_tile(3, 3)
        if difficult.lower() == "medium":
            self.slide_tile = Slide_tile(4, 3)
        if difficult.lower() == "hard":
            self.slide_tile = Slide_tile(4, 4)
        if difficult.lower() == "master":
            self.slide_tile = Slide_tile(5, 5)
        if self.verbose:
            print("GAME")
            print(self.slide_tile)
            print()
        # Types
        self.Tile = UserType("Tile")
        self.Position = UserType("Position")
        # Fluents
        self.is_tile = Fluent("is_tile", BoolType(), t=self.Tile)
        self.is_position = Fluent("is_position", BoolType(), p=self.Position)
        self.blank = Fluent("blank", BoolType(), bx=self.Position, by=self.Position)
        self.at = Fluent("at", BoolType(), t=self.Tile, px=self.Position, py=self.Position)
        self.dec = Fluent("dec", BoolType(), p1=self.Position, p2=self.Position)
        self.inc = Fluent("inc", BoolType(), p1=self.Position, p2=self.Position)
        # Problem
        self.problem = self.__create_problem()
        # Plan
        self.plan = self.__solve_problem()

    def __move_up(self):
        # SET VALUE
        up = InstantaneousAction("Move Up", t=self.Tile, px=self.Position, py=self.Position, bx=self.Position)
        t = up.parameter("t")
        px = up.parameter("px")
        py = up.parameter("py")
        bx = up.parameter("bx")
        # Preconditions
        up.add_precondition(self.is_tile(t))
        up.add_precondition(self.is_position(px))
        up.add_precondition(self.is_position(py))
        up.add_precondition(self.is_position(bx))
        up.add_precondition(self.inc(bx, px))
        up.add_precondition(self.blank(bx, py))
        up.add_precondition(self.at(t, px, py))
        # Effects
        up.add_effect(self.blank(bx, py), False)
        up.add_effect(self.at(t, px, py), False)
        up.add_effect(self.blank(px, py), True)
        up.add_effect(self.at(t, bx, py), True)
        return up

    def __move_down(self):
        down = InstantaneousAction("Move Down", t=self.Tile, px=self.Position, py=self.Position, bx=self.Position)
        # Parameters
        t = down.parameter("t")
        px = down.parameter("px")
        py = down.parameter("py")
        bx = down.parameter("bx")
        # Preconditions
        down.add_precondition(self.is_tile(t))
        down.add_precondition(self.is_position(px))
        down.add_precondition(self.is_position(py))
        down.add_precondition(self.is_position(bx))
        down.add_precondition(self.dec(bx, px))
        down.add_precondition(self.blank(bx, py))
        down.add_precondition(self.at(t, px, py))
        # Effects
        down.add_effect(self.blank(bx, py), False)
        down.add_effect(self.at(t, px, py), False)
        down.add_effect(self.blank(px, py), True)
        down.add_effect(self.at(t, bx, py), True)
        return down

    def __move_left(self):
        left = InstantaneousAction("Move Left", t=self.Tile, px=self.Position, py=self.Position, by=self.Position)
        # Parameters
        t = left.parameter("t")
        px = left.parameter("px")
        py = left.parameter("py")
        by = left.parameter("by")
        # Preconditions
        left.add_precondition(self.is_tile(t))
        left.add_precondition(self.is_position(px))
        left.add_precondition(self.is_position(py))
        left.add_precondition(self.is_position(by))
        left.add_precondition(self.inc(by, py))
        left.add_precondition(self.blank(px, by))
        left.add_precondition(self.at(t, px, py))
        # Effects
        left.add_effect(self.blank(px, by), False)
        left.add_effect(self.at(t, px, py), False)
        left.add_effect(self.blank(px, py), True)
        left.add_effect(self.at(t, px, by), True)
        return left

    def __move_right(self):
        right = InstantaneousAction("Move Right", t=self.Tile, px=self.Position, py=self.Position, by=self.Position)
        # Parameters
        t = right.parameter("t")
        px = right.parameter("px")
        py = right.parameter("py")
        by = right.parameter("by")
        # Preconditions
        right.add_precondition(self.is_tile(t))
        right.add_precondition(self.is_position(px))
        right.add_precondition(self.is_position(py))
        right.add_precondition(self.is_position(by))
        right.add_precondition(self.dec(by, py))
        right.add_precondition(self.blank(px, by))
        right.add_precondition(self.at(t, px, py))
        # Effects
        right.add_effect(self.blank(px, by), False)
        right.add_effect(self.at(t, px, py), False)
        right.add_effect(self.blank(px, py), True)
        right.add_effect(self.at(t, px, by), True)
        return right

    def __create_problem(self):
        # PROBLEM
        problem = Problem('problem')
        if self.optimal_plan:
            metric = MinimizeSequentialPlanLength()
            problem.add_quality_metric(metric)

        # FLUENT
        # Type checks
        problem.add_fluent(self.is_tile, default_initial_value=False)
        problem.add_fluent(self.is_position, default_initial_value=False)
        # Positions
        problem.add_fluent(self.blank, default_initial_value=False)
        problem.add_fluent(self.at, default_initial_value=False)
        # Connections
        problem.add_fluent(self.dec, default_initial_value=False)
        problem.add_fluent(self.inc, default_initial_value=False)

        # OBJECTS
        x_coordinates = []
        y_coordinates = []
        tiles = []
        # x-coordinates
        for i in range(self.slide_tile.n_row):
            x_coordinates.append(Object("x" + str(i), self.Position))
            problem.add_object(x_coordinates[-1])
        # y-coordinates
        for i in range(self.slide_tile.n_col):
            y_coordinates.append(Object("y" + str(i), self.Position))
            problem.add_object(y_coordinates[-1])
        # Tiles
        tiles_counter = 0
        for i in range(self.slide_tile.n_row):
            for j in range(self.slide_tile.n_col):
                if (i == self.slide_tile.n_row-1 and j == self.slide_tile.n_col-1):
                    continue
                tiles.append(Object("t" + str(tiles_counter), self.Tile))
                problem.add_object(tiles[-1])
                tiles_counter += 1

        # FLUENTS INIT
        # Position check for x-coordinates
        for x_coordinate in x_coordinates:
            problem.set_initial_value(self.is_position(x_coordinate), True)
        # Position check for y-coordinates
        for y_coordinate in y_coordinates:
            problem.set_initial_value(self.is_position(y_coordinate), True)
        # Position check for tiles
        for tile in tiles:
            problem.set_initial_value(self.is_tile(tile), True)
        # Connection increment x-coordinates
        for i, x_coordinate in enumerate(x_coordinates[:-1]):
            problem.set_initial_value(self.inc(x_coordinate, x_coordinates[i+1]), True)
        # Connection decrement x-coordinates
        for i, x_coordinate in enumerate(x_coordinates[1:]):
            problem.set_initial_value(self.dec(x_coordinate, x_coordinates[i]), True)
        # Connection increment y-coordinates
        for i, y_coordinate in enumerate(y_coordinates[:-1]):
            problem.set_initial_value(self.inc(y_coordinate, y_coordinates[i+1]), True)
        # Connection decrement y-coordinates
        for i, y_coordinate in enumerate(y_coordinates[1:]):
            problem.set_initial_value(self.dec(y_coordinate, y_coordinates[i]), True)
        # Tiles positions
        for i in range(self.slide_tile.n_row):
            for j in range(self.slide_tile.n_col):
                t = self.slide_tile.tiles[i][j]
                if t != "-":
                    problem.set_initial_value(self.at(tiles[int(t)],
                                                      x_coordinates[i],
                                                      y_coordinates[j]), True)
        # Tiles positions
        problem.set_initial_value(self.blank(x_coordinates[self.slide_tile.b_x],
                                             y_coordinates[self.slide_tile.b_y]), True)

        # ACTION
        problem.add_action(self.__move_up())
        problem.add_action(self.__move_down())
        problem.add_action(self.__move_right())
        problem.add_action(self.__move_left())

        # GOAL
        tiles_idx = 0
        for i in range(self.slide_tile.n_row):
            for j in range(self.slide_tile.n_col):
                if (i == self.slide_tile.n_row-1 and j == self.slide_tile.n_col-1):
                    continue
                problem.add_goal(self.at(tiles[tiles_idx],
                                         x_coordinates[i],
                                         y_coordinates[j]))
                tiles_idx += 1

        # PRINT
        if self.verbose:
            print("PROBLEM INITIALIZATION IN PDDL")
            print(problem)
            print()
        return problem

    def __solve_problem(self, timeout=False):
        opt = PlanGenerationResultStatus.SOLVED_OPTIMALLY if (self.optimal_plan and not timeout) else None
        with OneshotPlanner(problem_kind=self.problem.kind, optimality_guarantee=opt) as planner:
            if not timeout:
                if self.verbose:
                    print("Searching for optimal plan...")
                result = planner.solve(self.problem, timeout=TIMEOUT)
                if result.status == PlanGenerationResultStatus.TIMEOUT:
                    if self.verbose:
                        print("Optimal plan not fond in less than", TIMEOUT, "seconds")
                        print("Searching for non optimal plan...")
                    return self.__solve_problem(timeout=True)
            else:
                result = planner.solve(self.problem)
            if self.verbose:
                print(result)
            plan = result.plan
            if plan is None and self.verbose:
                print("No plan found.")
            return plan

    def check_plan(self):
        if self.plan is None:
            print("No plan to check")
        else:
            if self.verbose:
                print("\nCHECK THE PLAN")
                st = copy.deepcopy(self.slide_tile)
                for action in self.plan.actions:
                    move = str(action).split("(")[0].split(" ")[1]
                    print(move)
                    if move == "Up":
                        st.move_up()
                    elif move == "Down":
                        st.move_down()
                    elif move == "Right":
                        st.move_right()
                    else:
                        st.move_left()
                    print(st)
                    print()
            else:
                print("Verbose setted to False")

s = Slide_tile_PDDL("easy", optimal_plan=True)
s.check_plan()

In [232]:
# The execution time test function
def time_test(difficult):
    print(difficult)
    Slide_tile_PDDL(difficult, optimal_plan=True, verbose = False)

In [233]:
tempo_easy = timeit.timeit(lambda: time_test("easy"), number=3)

easy
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

easy
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

easy
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contribut

In [234]:
tempo_medium = timeit.timeit(lambda: time_test("medium"), number=3)

medium
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

medium
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

medium
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and con

In [235]:
tempo_hard = timeit.timeit(lambda: time_test("hard"), number=3)

hard
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

hard
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (

In [236]:
tempo_master = timeit.timeit(lambda: time_test("master"), number=3)

master
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

master
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 222 of `<ipython-input-222-ac6172ebf28d>`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributo

In [238]:
print("EXECUTION TIMES:")
print(f"\tExecution time at \"easy\": {round(tempo_easy, 3)} seconds")
print(f"\tExecution time at \"medium\": {round(tempo_medium, 3)} seconds")
print(f"\tExecution time at \"hard\": {round(tempo_hard, 3)} seconds")
print(f"\tExecution time at \"master\": {round(tempo_master, 3)} seconds")

EXECUTION TIMES:
	Execution time at "easy": 6.742 seconds
	Execution time at "medium": 6.387 seconds
	Execution time at "hard": 30.214 seconds
	Execution time at "master": 87.102 seconds
